# Khám phá dữ liệu

Tạm thời dừng việc làm sạch dữ liệu, ta chuyển sang xem có thể rút ra được những nhận xét gì từ bộ data này.

Trước hết cứ load lại dữ liệu

In [ ]:
# %load loaddata.py
import pandas as pd

pd.set_option('max.rows', 14)

data = pd.read_csv(
    'data/titanic.csv',
    index_col='PassengerId',
    usecols=lambda col: col not in ('Cabin', 'Ticket'),
    dtype={
        'Pclass': 'category',
        'Sex': 'category',
        'Embarked': 'category'
    }
)

data['Age'] = (data['Age'].fillna(data['Age'].mean())
                          .astype(int))
data['Embarked'].fillna(data['Embarked'].mode()[0], inplace=True)


In [ ]:
data.info()

Ngoài `info()` thì còn có hàm `describe()` mô tả cụ thể hơn

In [ ]:
data.describe()

Mặc định thì `describe()` chỉ mô tả các cột có kiểu dữ liệu số. Nếu muốn hiện hết thì có thể dùng parameter `include`

In [ ]:
data.describe(include='all')

Cũng có thể dùng `include` để chọn ra những loại dữ liệu cụ thể

In [ ]:
import numpy as np

data.describe(include=[np.number, 'category'])

### Exercise

Dùng `describe()` để mô tả riêng cột `Embarked`

In [ ]:
# Code vào đây

In [ ]:
%load solutions/describe_embarked.py

## Nhận biết các feature quan trọng

Câu hỏi quan trọng nhất trong bài toán này là "những yếu tố nào ảnh hưởng đến khả năng sống còn của 1 người trên tàu Titanic?". Để trả lời, ta có thể xem xét mối quan hệ giữa từng cột với cột `Survival`.
- `Pclass`, `Sex`, `Age` rõ ràng có ảnh hưởng.
- `Name`? Có thể thấy tên bao gồm cả _title_, ví dụ `Capt`, `Lady`... thể hiện địa vị xã hội của người đó. Ngoài ra có thể lấy ra họ của từng người, từ đó đoán thêm 1 số thông tin như sắc tộc, xuất xứ...
- Số lượng anh em, vợ/chồng `SibSp`? Bố mẹ con cái `Parch`? Có thể vì cứu nhau mà cùng sống hoặc cùng chết.
- Giá vé `Fare`? Nếu giá vé quyết định khoang ở thì có thể có ảnh hưởng, vì có khoang chìm trước khoang chìm sau.
- Điểm lên tàu `Embarked`? Tương tự `Fare`, có thể nó có ảnh hưởng đến khoang ở trên tàu.

Hầu hết các cột đã ở dạng rất dễ tham chiếu, chỉ có cột `Name` là có nhiều data thừa. Tạm không đụng đến họ mà chỉ quan tâm đến title, ta nên bóc tách phần thông tin cần thiết ra để dễ tham chiếu sau này.

## Feature engineering

Trước hết ta cần 1 function để lấy được title từ tên người. Regular Expression là perfect cho việc này. Thư viện Regular Expression trong Python là `re`

In [ ]:
import re

name_pattern = re.compile('.*, ([^.]*)\..*')

def get_title_from_name(name):
    m = name_pattern.match(name)
    if not m:
        raise ValueError(f'Name doesn\'t match pattern: {name}')
    return m.group(1)

# Test qua
for i in range(1, 10):
    print(get_title_from_name(data.loc[i, 'Name']))

Trông có vẻ ổn rồi. Giờ áp dụng trên toàn bộ cột `Name` để tạo ra cột mới `Title`

In [ ]:
data['Title'] = data['Name'].apply(get_title_from_name).astype('category')
data['Title']

Cột `Name` đã không còn giá trị, xóa

In [ ]:
data.drop(columns='Name')

Thử xem tần suất xuất hiện của từng title như thế nào

In [ ]:
with pd.option_context('max.rows', 20):
    print(data['Title'].value_counts())

Chú ý:
- `Mlle` (tiếng Pháp) = `Miss` (tiếng Anh), `Mme` (tiếng Pháp) = `Mrs` (tiếng Anh)
- Chỉ có 1 `Ms`, không đáng để để thành 1 category riêng, có thể ghép vào với `Miss`
- Ngoài `Mr`, `Miss`, `Mrs`, `Mlle`, `Mme`, `Ms` thì các title còn lại đều dành cho quý tộc/người được trọng vọng, và có tần suất xuất hiện rất thấp. Có thể gộp vào thành 1 category `High standing`

In [ ]:
# Đổi Mlle, Ms -> Miss, Mme -> Mrs
data.loc[(data['Title'] == 'Mlle') | (data['Title'] == 'Ms'), 'Title'] = 'Miss'
data.loc[data['Title'] == 'Mme', 'Title'] = 'Mrs'

with pd.option_context('max.rows', 20):
    print(data['Title'].value_counts())

Các category còn lại thì lằng nhằng hơn do pandas không có hàm merge category. Có thể xóa toàn bộ các category này để biến các value tương ứng bị `null`, rồi điền `High standing` vào tất cả các chỗ `null`

In [ ]:
data['Title'].cat.set_categories(('Mr', 'Mrs', 'Miss', 'High Standing'), inplace=True)
data['Title'].fillna('High Standing', inplace=True)
data['Title']